In [ ]:
strain = "BY4741"
paralog = "FKS1"
# Chain identifier in the PDB
chain = "/F"

hotspots = ["HS1", "HS2", "HS3"]
compound = "caspofungin"

# Notebook to generate defattr files for ChimeraX

These files are later imported in ChimeraX to plot DMS data on protein structures

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

## Enter constants

In [ ]:
# Color palette
palette = "#F1F1F1:#E4C1D9:#D691C1:#C75DAB"

## Retrieve all classified data

In [ ]:
list_df = []

for hs in hotspots:
    locus = "-".join([paralog, hs])
    hs_df = pd.read_csv(f"../heatmap_data/{strain}_{locus}_{compound}.csv")
    hs_df["Hotspot"] = hs
    list_df.append(hs_df)

df = pd.concat(list_df, ignore_index=True)

## Aggregate

In [ ]:
attrdf = (
    df.groupby("aa_pos")[["sensres"]]
    .agg(lambda x: x[x.str.contains("resistant")].count() / len(x))
    .reset_index()
)
attrdf["pos"] = attrdf["aa_pos"].apply(lambda x: f"{chain}:{str(int(x))}")
attrdf["empty_col"] = ""
attrdf

## Convert to defattr file

In [ ]:
custom_header = (
    f"#\nattribute: res_{compound}\nmatch mode: 1-to-1\nrecipient: residues\n"
)
with open(f"../defattr/{strain}_{paralog}_{compound}_defattr.txt", "w") as fp:
    fp.write(custom_header)

attrdf[["empty_col", "pos", "sensres"]].to_csv(
    f"../defattr/{strain}_{paralog}_{compound}_defattr.txt",
    sep="\t",
    header=None,
    index=None,
    mode="a",
)

cmd = f"color byattribute res_{compound} range 0, 1 palette {palette}"
print("Command lines to enter in ChimeraX")
print(f'open "../defattr/{strain}_{paralog}_{compound}_defattr.txt" format defattr')
print(cmd)

In [ ]:
# Example of custom color key
# Here, proportion of resistant mutants
print(
    f'key {palette.split(":")[0]}:0% '
    + " ".join([f"{c}:" for c in palette.split(":")[1:-1]])
    + f' {palette.split(":")[-1]}:100%'
)